In [1]:
data_root = '/home/scott/Documents/SPEC_Spider'

In [2]:
!tree /home/scott/Documents/SPEC_Spider

/home/scott/Documents/SPEC_Spider
├── cpu
│   ├── cpu2006
│   │   ├── SPECfp.csv
│   │   ├── SPECfp_rate.csv
│   │   ├── SPECint.csv
│   │   └── SPECint_rate.csv
│   └── cpu2017
│       ├── CFP2017_rate.csv
│       ├── CFP2017_speed.csv
│       ├── CINT2017_rate.csv
│       └── CINT2017_speed.csv
├── java
│   ├── jbb2015
│   │   ├── SPECjbb2015-Composite.csv
│   │   ├── SPECjbb2015-Distributed.csv
│   │   └── SPECjbb2015-MultiJVM.csv
│   └── jvm2008
│       └── jvm2008.csv
├── jbb2015.csv
└── power
    └── ssj2008.csv

7 directories, 14 files


In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
c2006_rfp = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECfp_rate.csv")
c2006_sfp = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECfp.csv")
c2006_rint = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECint_rate.csv")
c2006_sint = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECint.csv")

In [5]:
cpu2006_columns = [
    'Suite', 'Hardware Vendor', 'System Name','Baseline',
    'Test Date', 'HW Avail', 'CPU Name', 'CPU Characteristics',
    'CPU MHz', 'CPU(s) enabled', 'CPU(s) orderable', 'Primary Cache',
    'Secondary Cache', 'L3 Cache', 'Memory', 'Disk Subsystem',
    'OS', 'File System', 'URL Suffix',
]
c2006_rfp = c2006_rfp[cpu2006_columns]
c2006_sfp = c2006_sfp[cpu2006_columns]
c2006_rint = c2006_rint[cpu2006_columns]
c2006_sint = c2006_sint[cpu2006_columns]

In [6]:
c2006 = pd.concat([c2006_rfp, c2006_sfp, c2006_rint, c2006_sint]).reset_index(drop=True)

In [7]:
c2006_rename_dict = {
    'Hardware Vendor': 'HW Vendor',
    'CPU(s) enabled': 'CPU Enabled',
    'CPU(s) orderable': 'CPU Orderable',
    'Primary Cache': 'L1 Cache',
    'Secondary Cache': 'L2 Cache',
    'L3 Cache': 'L3 Cache',
    'Disk Subsystem': 'Storage',
}
c2006.rename(columns=c2006_rename_dict, inplace=True)

In [8]:
c2006.shape

(48381, 19)

In [9]:
c2006 = c2006[~c2006.isnull().any(axis=1)].reset_index(drop=True)

In [10]:
c2006.columns

Index(['Suite', 'HW Vendor', 'System Name', 'Baseline', 'Test Date',
       'HW Avail', 'CPU Name', 'CPU Characteristics', 'CPU MHz', 'CPU Enabled',
       'CPU Orderable', 'L1 Cache', 'L2 Cache', 'L3 Cache', 'Memory',
       'Storage', 'OS', 'File System', 'URL Suffix'],
      dtype='object')

## Clean Vendor and System Name

In [11]:
def clean_vendor(vendor):
    def _clear(pattern):
        return re.sub(re.compile(pattern, re.IGNORECASE), '', vendor)
    vendor = vendor.strip()
    patterns = [
        ' *[(].*[)]',
        ',* *Ltd\.*$|,* *Inc\.*$',
        ',* *Co\.*$|,* *Corporation\.*$|,* *Corparation\.*$|,* *Corp\.*$| Incoporated$| Incorporated$| Incorporation$',
        ' International$',
        ' Computer[s]*$',
        ' Technology$'
    ]
    for pattern in patterns:
        vendor = _clear(pattern)
    
    replace_pairs = [
        ('^Huawei', 'Huawei'),
        ('^ASUS', 'ASUS'),
        ('^acer', 'Acer'),
        ('^Hewlett[ -]*Packard', 'HPE'),
        ('^Inspur', 'Inspur'),
        ('H3C', 'H3C'),
        ('^Giga[ -]*byte', 'Gigabyte'),
        ('^Fujitsu', 'Fujitsu'),
        ('^Hitachi', 'Hitachi'),
        ('^Lenovo', 'Lenovo'),
        ('^Quanta', 'Quanta'),
        ('^Super[ -]*Micro', 'SuperMicro'),
        ('^UNIWIDE', 'Uniwide'),
        ('^Wizbrain', 'Wizbrain'),
        ('^ScaleMP', 'ScaleMP'),
        ('^AMD', 'AMD'),
        ('Advanced Micro Devices', 'AMD'),
        ('^Hewelett-Packard', 'HPE'),
        ('^Oracl', 'Oracle'),
        ('^BEA', 'BEA'),
        ('^OpenJDK', 'OpenJDK'),
    ]
    for pair in replace_pairs:
        if len(re.findall(re.compile(pair[0], re.IGNORECASE), vendor)):
            vendor = pair[1]
    
    return vendor

In [12]:
c2006['HW Vendor'] = c2006['HW Vendor'].apply(lambda x: clean_vendor(x))

In [13]:
def parse_system_name(info):
    info = re.sub('[(].*[)]', '', info)
    info = re.sub('[(]|[)]', '', info)
    info = info.strip()
    info = info.split(',')[0]
    info = re.sub(' AMD.*?', '', info)
    info = re.sub(' Intel.*', '', info)
    info = re.sub('\d+.*\d*GHz$', '', info)
    info = re.sub('^vSMP ServerONE Supermicro ', '', info)
    info = info.strip()
    return info
c2006['System Name'] = c2006['System Name'].apply(lambda x: parse_system_name(x))

## Clean CPU

In [14]:
c2006.columns

Index(['Suite', 'HW Vendor', 'System Name', 'Baseline', 'Test Date',
       'HW Avail', 'CPU Name', 'CPU Characteristics', 'CPU MHz', 'CPU Enabled',
       'CPU Orderable', 'L1 Cache', 'L2 Cache', 'L3 Cache', 'Memory',
       'Storage', 'OS', 'File System', 'URL Suffix'],
      dtype='object')

In [15]:
def get_cpu_vendor(cpu_name):
    item = cpu_name.split()[0]
    if item in ['Intel', 'AMD', 'Huawei']:
        vendor = item
    else:
        vendor = 'Other'
    return vendor
c2006['CPU Vendor'] = c2006['CPU Name'].apply(lambda x: get_cpu_vendor(x))

In [19]:
c2006['CPU MHz'] = c2006['CPU MHz'].apply(lambda x: round(x /1000, 2))

In [34]:
re.findall('\d+.*?\d*[ ]*G[ ]*Hz', 'Intel Turbo Boost Technology up to 2.40G Hz') 
re.findall('\d+\.*\d*[ ]?G[ ]?Hz', '3.0GHz 1333MHz System Bus')

['3.0GHz']

In [38]:
def parse_cpu_char(info):
    items = re.findall('\d+\.*\d*[ ]?G[ ]?Hz', info)
    if len(items) == 0:
        items = re.findall('\d+\.*\d*[ ]?M[ ]?Hz', info)
        if len(items) == 0:
            value = 0
        else:
            value = float(re.sub('[ ]?M[ ]?Hz', '', items[0]))
    else:
        value = float(re.sub('[ ]?G[ ]?Hz', '', items[0]))
    return value
c2006['Max MHz'] = c2006['CPU Characteristics'].apply(lambda x: parse_cpu_char(x))

In [40]:
f = c2006['Max MHz'] == 0.
indices = c2006[f].index
cpu_mhz = c2006.loc[indices, 'CPU MHz']
c2006.loc[indices, 'Max MHz'] = c2006.loc[indices, 'CPU MHz']

In [43]:
c2006.columns

Index(['Suite', 'HW Vendor', 'System Name', 'Baseline', 'Test Date',
       'HW Avail', 'CPU Name', 'CPU Characteristics', 'CPU MHz', 'CPU Enabled',
       'CPU Orderable', 'L1 Cache', 'L2 Cache', 'L3 Cache', 'Memory',
       'Storage', 'OS', 'File System', 'URL Suffix', 'CPU Vendor', 'Max MHz'],
      dtype='object')

In [46]:
def parse_cpu_enabled(info):
    items = info.split(',')
    threads_per_core = 1
    total_cores = int(items[0].split()[0])
    chips = int(items[1].split()[0])
    cores_per_chip = int(items[2].split()[0])
    if len(items) == 4:
        cores_per_chip = (items[-1].split()[0])
    return pd.Series({
        'Total Cores': total_cores,
        'Chips': chips,
        'Cores Per Chip': cores_per_chip,
        'Threads Per Core': threads_per_core,
    })
c2006[['Total Cores', 'Chips', 'Cores Per Chip', 'Threads Per Core']] = c2006['CPU Enabled'].apply(lambda x: parse_cpu_enabled(x))

In [59]:
def parse_cpu_orderable(info):
    info = info.split(';')[0]
    items = re.findall('.*chip', info)
    number = 1
    if len(items):
        nums = [int(num) for num in re.findall(r'\d+', items[0])]
        number = nums[-1]
    return number
c2006['Max Chips'] = c2006['CPU Orderable'].apply(lambda x: parse_cpu_orderable(x))

In [60]:
c2006['Max Chips'].unique()

array([   2,    4,    1,    8,   32,  128,  256,   64,  512,   16, 9999,
         96])

In [62]:
c2006['File System'] = c2006['File System'].apply(lambda x: x.lower())

In [66]:
c2006['Memory'].unique()

array(['128 GB (8 x 16 GB 2Rx4 PC3-10600R-9 ECC)',
       '4 TB (128 x 32 GB 2Rx4 PC4 - 2400T, running at 1600)',
       '8 TB (128 x 64 GB 4Rx4 PC4 - 2400T, running at 1600)', ...,
       '256 GB (16 x 16 GB 2Rx4 PC4-1866P-R)',
       '96 GB (12 x 8 GB 2Rx4 PC3-12800R-11, ECC, running at 1333 Mhz and CL9)',
       '24 GB (6 x 4GB DDR3-1066 RDIMM, CL9)'], dtype=object)

In [71]:
def parse_memory(info):
    total_memory_amount = int(info.split()[0])
    unit = info.split()[1]
    if 'T' in unit:
        total_memory_amount *= 1024
    items = re.findall(r'\d+ x', info)
    nums = [int(item.split()[0]) for item in items]
    memory_num = sum(nums)
    if memory_num == 0:
        memory_num = 1
    return pd.Series({
        'Total Memory Amount': total_memory_amount,
        'Memory Number': memory_num,
    })
c2006[['Total Memory Amount', 'Memory Number']] = c2006['Memory'].apply(lambda x: parse_memory(x))  

In [72]:
c2006['Memory Amount'] = c2006.apply(lambda item: item['Total Memory Amount'] // item['Memory Number'], axis=1)

In [75]:
c2006['Storage'].unique()

array(['1 x 300 GB SAS, 10K RPM', '1 x 7.3 TB 7200 RPM SATA',
       '2 x 2.9 TB NVMe SSD', ..., '1 x 600GB SAS, 15K RPM',
       '1 X 300 GB 10000 RPM SAS Disk', '1 x 10000 GB SATA, 7200 RPM'],
      dtype=object)

In [76]:
def parse_storage(info):
    # items = re.findall(r'\d+[ ]*x', info)
    # storage_num = 1
    # if len(items) != 0:
    #     storage_num = items[0][:-1].strip()
    # storage_num = int(storage_num)
    # items = re.findall(r'\d+[.]?\d*[ ]*TB?', info)
    # if len(items):
    #     storage_size = float(re.findall(r'\d+\.?\d*', items[0])[0])
    #     storage_size = storage_size * 1024
    # else:
    #     items = re.findall(r'\d+[.]*\d*[ ]*GB?', info)
    #     storage_size = float(re.findall(r'\d+\.?\d*', items[0])[0])
    if 'SSD' in info.upper():
        storage_type = 'SSD'
    elif 'HDD' in info.upper():
        storage_type = 'HDD'
    elif 'ramfs' in info.lower():
        storage_type = 'ramfs'
    elif 'tmpfs' in info.lower():
        storage_type = 'tmpfs'
    elif 'zfs' in info.lower():
        storage_type = 'zfs'
    else:
        storage_type = 'SSD'
    return pd.Series({
        # 'Storage Number': storage_num,
        # 'Storage Size': storage_size,
        'Storage Type': storage_type
    })

c2006['Storage Type'] = c2006['Storage'].apply(lambda x: parse_storage(x))

## Clean Date

In [77]:
month_mapper = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May':5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}
re_month_mapper = {v:k for k, v in month_mapper.items()}

re_month_mapper

{1: 'Jan',
 2: 'Feb',
 3: 'Mar',
 4: 'Apr',
 5: 'May',
 6: 'Jun',
 7: 'Jul',
 8: 'Aug',
 9: 'Sep',
 10: 'Oct',
 11: 'Nov',
 12: 'Dec'}

In [78]:
def split_date_1(item):
    """
    May-2017
    2014.3
    2010
    """
    item = item.strip()
    d = 1
    if '-' in item:
        m, y = item.split('-')
    elif '.' in item:
        y, m = item.split('.')
        m = re_month_mapper[int(m)]
    elif ' ' in item:
        m, y = item.split()
    else:
        y = item
        m = 'Jan'
    m = month_mapper[m[:3]]
    y = int(y)
    return pd.Series({'day': d, 'month': m, 'year': y})

In [79]:
def split_date_2(item):
    """
    May 1, 2018
    2009/05/19
    12.02.2009
    """
    if ',' in item:
        md, y = item.split(',')
        m, d = md.split()
    elif '/' in item:
        y, m, d = item.split('/')
        m = re_month_mapper[int(m)]
    elif '.' in item:
        d, m, y = item.split('.')
        m = re_month_mapper[int(m)]
    m = m[:3]
    if m == 'Spe':
        m = 'Sep'
    m = month_mapper[m[:3]]
    d, m, y = int(d), int(m), int(y)
    return pd.Series({'day': d, 'month': m, 'year': y})

In [80]:
def get_quarter(url):
    items = url.split('/')[0]
    return pd.Series({'quarter': int(items[-1]), 'submit_year': int(items[3:7])})

In [81]:
c2006.columns

Index(['Suite', 'HW Vendor', 'System Name', 'Baseline', 'Test Date',
       'HW Avail', 'CPU Name', 'CPU Characteristics', 'CPU MHz', 'CPU Enabled',
       'CPU Orderable', 'L1 Cache', 'L2 Cache', 'L3 Cache', 'Memory',
       'Storage', 'OS', 'File System', 'URL Suffix', 'CPU Vendor', 'Max MHz',
       'Total Cores', 'Chips', 'Cores Per Chip', 'Threads Per Core',
       'Max Chips', 'Total Memory Amount', 'Memory Number', 'Memory Amount',
       'Storage Type'],
      dtype='object')

In [82]:
c2006[['day', 'month', 'year']] = c2006['Test Date'].apply(lambda x: split_date_1(x))

In [83]:
c2006[['quarter', 'submit_year']] = c2006['URL Suffix'].apply(lambda x: get_quarter(x))

In [84]:
c2006.columns

Index(['Suite', 'HW Vendor', 'System Name', 'Baseline', 'Test Date',
       'HW Avail', 'CPU Name', 'CPU Characteristics', 'CPU MHz', 'CPU Enabled',
       'CPU Orderable', 'L1 Cache', 'L2 Cache', 'L3 Cache', 'Memory',
       'Storage', 'OS', 'File System', 'URL Suffix', 'CPU Vendor', 'Max MHz',
       'Total Cores', 'Chips', 'Cores Per Chip', 'Threads Per Core',
       'Max Chips', 'Total Memory Amount', 'Memory Number', 'Memory Amount',
       'Storage Type', 'day', 'month', 'year', 'quarter', 'submit_year'],
      dtype='object')

In [85]:
c2006['HW Avail'].unique()

array(['Sep-2013', 'Dec-2017', 'Jun-2013', 'Apr-2011', 'Jul-2017',
       'Jun-2016', 'Mar-2017', 'Mar-2016', 'Oct-2015', 'Nov-2015',
       'Apr-2016', 'Dec-2015', 'Sep-2015', 'Aug-2015', 'Sep-2014',
       'May-2014', 'Jun-2015', 'Nov-2014', 'Jan-2015', 'Oct-2014',
       'Dec-2013', 'Aug-2013', 'Mar-2014', 'Oct-2013', 'Jan-2014',
       'Jul-2013', 'Nov-2012', 'Mar-2012', 'Feb-2012', 'Nov-2011',
       'Feb-2013', 'Dec-2012', 'Sep-2012', 'Apr-2012', 'Jan-2012',
       'May-2012', 'Mar-2011', 'Jun-2012', 'Oct-2011', 'Aug-2011',
       'Jun-2011', 'Jan-2010', 'Feb-2011', 'Apr-2010', 'Mar-2010',
       'Jun-2010', 'May-2010', 'Sep-2009', 'Oct-2009', 'Nov-2008',
       'Nov-2007', 'Sep-2007', 'Jul-2007', 'May-2007', 'Nov-2006',
       'Dec-2006', 'May-2006', 'Apr-2007', 'Jan-2007', 'Apr-2009',
       'Jan-2009', 'Feb-2008', 'Oct-2007', 'May-2016', 'May-2015',
       'Jun-2014', 'Feb-2014', 'Nov-2013', 'Mar-2013', 'Dec-2014',
       'Mar-2009', 'Mar-2008', 'Jan-2008', 'Aug-2007', 'Nov-20

In [86]:
c2006['HW Avail'].apply(lambda x: split_date_1(x))`

SyntaxError: invalid syntax (2501212078.py, line 1)